## Scope

Use case:
Plot cells from 1 or more CosMx FOVs as polygons and color cells using quantitative or qualitative attributes.

Inputs: 
{slide}-polygons.csv

Anndata (h5ad) file, e.g. exported from Seurat, containing metadata (adata.metadata) and counts (adata.X) for the FOVs of interest. 
If no anndata is parsed, one can plot the cells with random colors to visualise the segmentation

Output: image of cells as polygons

## Instructions

Use the different plotting routines depending on whether the cells are colored by metadata vs count matrix, and qualitative vs quantitative variables.

In [6]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
import seaborn as sns
import pandas as pd
import os
import random
import anndata as ad
#import squidpy as sq

In [29]:

hd5f_path = "/mnt/share/Projects/Proj_AN1_P.chabaudi/Datasets/CosmX/spleen/spleen_sel_v2.h5ad"
adata = ad.read_h5ad(hd5f_path)
adata

outdir = "/mnt/share/Projects/Proj_AN1_P.chabaudi/Datasets/CosmX/spleen/IF_images/"

poly= pd.read_csv(poly_csv)

/home/adnane/miniforge3/envs/scrnaseq3.10/lib/python3.10/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [76]:
slide = "TOM1" # this is the name of the folder in which the hd5f was exported from Seurat
poly_csv = "/mnt/share/Projects/Proj_AN1_P.chabaudi/Datasets/CosmX/ThomasOttoMouseSlide1060324/ThomasOttoMouseSlide1060324-polygons.csv"
sample = ["TPN1_spleen", "TP2_spleen"][0]
fovs_to_sel_tpn1 = [174, 175, 176, 177, 181, 182, 183 , 189 , 190, 196, 197, 203 ]
fovs_to_sel_tp2 = [67, 68, 69, 78, 79, 80, 89 , 90 , 91, 100, 101, 102 ]
fovs_to_sel_slide = fovs_to_sel_tp2

"""
slide = "TOM2" # this is the name of the folder in which the hd5f was exported from Seurat
poly_csv = "/mnt/share/Projects/Proj_AN1_P.chabaudi/Datasets/CosmX/ThomasOttoMouseSlide2060324/ThomasOttoMouseSlide2060324-polygons.csv"
sample = ["TP3_spleen","TP4_spleen"][0]
fovs_to_sel_tp3 = [81, 82, 83, 84, 85, 93, 94, 95, 96, 97, 104, 105 ]
fovs_to_sel_tp4 = [143, 144, 145, 146, 147, 151, 152, 161, 162, 171, 172, 173 ]
fovs_to_sel_slide = fovs_to_sel_tp3
"""
# In case you don't want to render the whole slide, select FOVs - 
# these should be named as in the original csv output, i.e. only numbers, not string
# else set : fovs_to_sel = []
fovs_to_sel = fovs_to_sel_slide

In [77]:
if len(fovs_to_sel) > 0:
    fov = poly[poly["fov"].isin(fovs_to_sel) ]
else:
    fov = poly

cells = fov["cell"].unique()
minx = fov.min()["x_global_px"]
maxx = fov.max()["x_global_px"]
miny = fov.min()["y_global_px"]
maxy = fov.max()["y_global_px"]
print("Drawing this many cells: ", len(cells))

Drawing this many cells:  36127


In [78]:
spleen_cluster_ids_v4 = {
  0:"Cluster0", #0
  1:"Erythrocytes",# 1 : Hbb,  Hba-a1/2
  2:"NK cells",# 2 : Klrb1, Mmp9
  3:"Marginal zone",# 3 :Hmgb2, Ppia
  4:"B cells group4+5",# 4: Igha
  5:"B cells group1+2+3",# 5: Cd74, Cd37, Cd79a, Ms4a1
  6:"Macrophages",# 6: Psap, C1qb, C1qa
  7:"T cells",# 7: Ccl5, Cd3g, Cd3d, Cd3e, Ms4a4b
  8:"Fibroblasts",# 8: Col1a1, Col1a2, Ccl19
  9:"B cells group1+2+3",# 9:Ighm, Igkc, Jchain, Hsp90b1, Xbp1, Mzb1 CLUMPING
  10:"Neutrophils",# 10: S100a8, S100a9, Camp
  11:"B cells group1+2+3",# 11: Igkc, Jchain, Hsp90b1, Xbp1, Mzb1
  12:"B cells group4+5",# 12: Ighg1
  13:"B cells group1+2+3",# 13: Igkc, Slpi, Jchain
  14:"Platelets",# 14: Pf4
  15:"Neutrophils",# 15: Mpo, Elane, Prtn3
  16:"Neutrophils",# 16: Mpo, Elane, Prtn3
  17:"Erythrocytes/Macrophages",# 17 : Hbb,  Hba-a1/2
  18:"Neutrophils",# 18: S100a8, S100a9, Camp
  19:"Erythrocytes",# 19 : Hbb
  20:"B cells group1+2+3",# 20: Igkc, Slpi, Jchain
  21:"B cells group1+2+3"# 21: Igha
}

adata.obs["celltypes_manual_v4"] = (adata.obs["harmony_clusters"].map(spleen_cluster_ids_v4).astype('category'))

In [79]:
# USE THIS FOR CATEGORICAL VALUES FROM ADATA METADATA
# cells that were filtered out during QC are displayed in black, to distinguish 'gaps' in tissue where no cell was seen or segmented
# from actual cells

var = "celltypes_manual_v4" #"seurat_clusters" "celltypes_manual"
# In case you want to display only a subset of categories and render the rest as other (different value)
allcats = {}
allcats["general"] = ["Erythrocytes","Platelets","Erythrocytes/Macrophages","Fibroblasts"]

allcats["foll"] = ["Cluster0", "Neutrophils","NK cells", "Marginal zone",  "Macrophages", "T cells" , "B cells group1+2+3", "B cells group4+5"]
allcats["all"] = []
plotcat = "foll"
cats = allcats[plotcat]
img_out= f"{outdir}{sample}_polygons_{var}_{plotcat}.png"
 


patches = []
fig, ax = plt.subplots()
ax.axis([minx,maxx,miny,maxy])

if len(cats) == 0 :
    cats = [str(r) for r in adata.obs[var].unique()]
cols = dict(zip(cats, sns.color_palette("hls", n_colors = len(cats))))
cols["failed QC"] = "black"
cols["other"] = "grey"

for cell in cells:
    df = fov[fov["cell"] == cell]
    #print(df.shape)
    xy = df[["x_global_px", "y_global_px"]].to_numpy()
    #print(xy)
    try :
        label = str(adata.obs.loc[cell][var])
        if label not in cats : 
            label = "other"      
    except:
        label = "failed QC"
    col = cols[label]

    #print(label, col)
    shape = Polygon(xy, fill = True, facecolor = col, edgecolor = col, alpha = 0.8, linewidth=0.1, label = label)
    ax.add_patch(shape)

    # needs a speed up
    #shape = Polygon(xy, fill = True, facecolor = col, edgecolor = col, alpha = 0.8, linewidth=0.1, label = label)
    #patches.append(shape)

#collection = PatchCollection(patches)
#ax.add_collection(collection)

markers = [plt.Line2D([0,0], [0,0], color = cols[item], alpha = 0.8, marker = 'o', linestyle = '') for item in cols]
ax.legend(markers, cols.keys(), bbox_to_anchor = (1,0.5))
print("Done")


In [ ]:
# USE THIS FOR CONTINUOUS VARIABLES WITH VALUES DERIVED FROM METADATA TABLE (immunofluorescence, count statistics..)

var = "Mean.DAPI"  # "nCount_RNA","Mean.CD298.B2M", "Mean.PanCK",'Mean.DAPI', "Mean.CD45", 'Mean.CD3'
norm = "linear" #"log"

img_out= f"{outdir}{sample}_polygons_"+ var +".png"

patches = []
fig, ax = plt.subplots()
ax.axis([minx,maxx,miny,maxy])

for cell in cells:
    df = fov[fov["cell"] == cell]

    xy = df[["x_global_px", "y_global_px"]].to_numpy()
    shape = Polygon(xy, fill = True) #, facecolor = col, edgecolor = col, alpha = 0.5, linewidth=0.1)
    patches.append(shape)

collection = PatchCollection(patches, cmap="turbo", norm = norm, alpha = 0.5, linewidth=0.1 )# match_original = true )
collection.set_array(adata.obs[var])
ax.add_collection(collection)

fig.colorbar(collection,  fraction=0.03, pad = 0.04)
print("Done")

In [ ]:
# USE THIS FOR CONTINUOUS VARIABLES WITH GENES FROM PANEL (values fetched from adata.X)

var = "C1qa"  # "Hba1_a2_b", "Cd79a"
norm = "linear" #"log"

img_out= f"{outdir}{sample}_polygons_"+ var +".png"

patches = []
fig, ax = plt.subplots()
ax.axis([minx,maxx,miny,maxy])

row = adata.var_names.get_loc(var)
expr = adata.X[row, :]

for cell in cells:
    df = fov[fov["cell"] == cell]

    xy = df[["x_global_px", "y_global_px"]].to_numpy()
    shape = Polygon(xy, fill = True) #, facecolor = col, edgecolor = col, alpha = 0.5, linewidth=0.1)
    patches.append(shape)

collection = PatchCollection(patches, cmap="turbo", norm = norm, alpha = 0.5, linewidth=0.1 )# match_original = true )
collection.set_array(expr)
ax.add_collection(collection)

fig.colorbar(collection,  fraction=0.03, pad = 0.04)
print("Done")

In [75]:
ax.set_title(f"{sample} - {var}")
ax.set_aspect('equal', 'box')

plt.show()

fig.savefig(img_out, format="png", dpi=900, bbox_inches = "tight")